In [ ]:
import dask
dask.config.set({"dataframe.query-planning": False})

import pandas as pd
import sys
import matplotlib.pyplot as plt

sys.path.append("../../../workflow/scripts/")
import readwrite
cfg = readwrite.config()

import geopandas as gpd
import shapely.geometry as sg
from shapely.strtree import STRtree
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from shapely.validation import make_valid

def plot_polygon(ax, geometry, color, alpha=0.5):
    """Plots a Shapely geometry (Polygon, MultiPolygon, or Point) on a matplotlib Axes."""
    if geometry.is_empty:
        return  # Skip plotting empty geometries

    if geometry.geom_type == 'Polygon':
        patches = [Polygon(geometry.exterior.coords, closed=True)]
        collection = PatchCollection(patches, facecolor=color, edgecolor=color, alpha=alpha)
        ax.add_collection(collection)

    elif geometry.geom_type == 'MultiPolygon':
        patches = []
        for p in geometry.geoms:
            patches.append(Polygon(p.exterior.coords, closed=True))
        collection = PatchCollection(patches, facecolor=color, edgecolor=color, alpha=alpha)
        ax.add_collection(collection)

    elif geometry.geom_type == 'Point':
        ax.plot(geometry.x, geometry.y, marker='o', color=color, markersize=5, alpha=alpha)

    elif geometry.geom_type == 'GeometryCollection':
        # Plot the individual geometries within the collection
        for geom in geometry.geoms:
            plot_polygon(ax, geom, color, alpha) # Recursive call to plot each geometry

    else:
        print(f"Skipping unsupported geometry type: {geometry.geom_type}")
        return  # Skip plotting if it's not a supported type

    ax.autoscale_view()



def plot_polygon(ax, geometry, color, alpha=0.5):
    """Plots a Shapely geometry (Polygon, MultiPolygon, or Point) on a matplotlib Axes."""
    if geometry.is_empty:
        return  # Skip plotting empty geometries

    if geometry.geom_type == 'Polygon':
        patches = [Polygon(geometry.exterior.coords, closed=True)]
        collection = PatchCollection(patches, facecolor=color, edgecolor=color, alpha=alpha)
        ax.add_collection(collection)

    elif geometry.geom_type == 'MultiPolygon':
        patches = []
        for p in geometry.geoms:
            patches.append(Polygon(p.exterior.coords, closed=True))
        collection = PatchCollection(patches, facecolor=color, edgecolor=color, alpha=alpha)
        ax.add_collection(collection)

    elif geometry.geom_type == 'Point':
        ax.plot(geometry.x, geometry.y, marker='o', color=color, markersize=5, alpha=alpha)

    elif geometry.geom_type == 'GeometryCollection':
        # Plot the individual geometries within the collection
        for geom in geometry.geoms:
            plot_polygon(ax, geom, color, alpha) # Recursive call to plot each geometry

    else:
        print(f"Skipping unsupported geometry type: {geometry.geom_type}")
        return  # Skip plotting if it's not a supported type

    ax.autoscale_view()

def intersecting_geometries_shapely(gdf1, gdf2,overlap_only=True):
    """
    Finds the intersections between all pairs of geometries from gdf1 and gdf2,
    labels them with the 'cell' identifiers, and returns the results in a DataFrame.
    Handles MultiPolygons directly without expansion.

    Args:
        gdf1 (geopandas.GeoDataFrame): The GeoDataFrame containing the geometries to check for intersections.
                                        Must have a 'geometry' column and a 'cell' column.
        gdf2 (geopandas.GeoDataFrame): The GeoDataFrame containing the geometries to use as search geometries.
                                        Must have a 'geometry' column and a 'cell' column.
        overlap_only (bool): Only keep ['Polygon', 'MultiPolygon'] geometries and not ['LineString', 'Point']

    Returns:
        pandas.DataFrame: A DataFrame containing the intersections, labeled with the 'cell' identifiers
                          from both GeoDataFrames. The DataFrame has columns 'cell1', 'cell2', and 'intersection'.
                          Returns an empty DataFrame if there are no intersections.
    """

    if not all(col in gdf1.columns for col in ['geometry', 'cell']):
        raise ValueError("gdf1 must have 'geometry' and 'cell' columns")
    if not all(col in gdf2.columns for col in ['geometry', 'cell']):
        raise ValueError("gdf2 must have 'geometry' and 'cell' columns")


    gdf1['geometry'] = gdf1['geometry'].apply(lambda geom: make_valid(geom))
    gdf2['geometry'] = gdf2['geometry'].apply(lambda geom: make_valid(geom))

    # 1. Prepare geometries and cell labels
    geometries1 = list(gdf1['geometry'])
    cell_labels1 = list(gdf1['cell'])

    geometries2 = list(gdf2['geometry'])
    cell_labels2 = list(gdf2['cell'])

    # 2. Build STRtree using geometries from gdf1
    tree = STRtree(geometries1)

    # 4. Perform the bulk query - all geometries from gdf2 at once
    possible_intersections_idx = tree.query(geometries2)

    # 5.  Process possible intersections
    intersection_data = []
    for i in range(len(possible_intersections_idx[0])):
      geom_index = possible_intersections_idx[1][i]
      geom_index2 = possible_intersections_idx[0][i]

      if geom_index < len(geometries1) and geom_index2 < len(geometries2): #Validate indexes to check

        geom1 = geometries1[geom_index]  # From gdf1
        geom2 = geometries2[geom_index2]  # From gdf2
        cell1 = cell_labels1[geom_index]
        cell2 = cell_labels2[geom_index2]

        if geom1.geom_type != 'Point' and geom2.geom_type != 'Point':  # Skip Point intersections

            intersection = geom2.intersection(geom1)  # Calculate intersection
            if not intersection.is_empty and intersection.geom_type in ['Polygon', 'MultiPolygon']:
                intersection_data.append({
                    'cell1': cell1,
                    'cell2': cell2,
                    'intersection': intersection
                })

    # 6.  Create a DataFrame from the intersection data
    df_intersection = pd.DataFrame(intersection_data)
    return df_intersection


def intersecting_geometries(gdf1, gdf2,overlap_only=True,):
    """
    Finds areal overlaps between two GeoDataFrames using GeoPandas' sjoin.

    Args:
        gdf1 (gpd.GeoDataFrame): The first GeoDataFrame.
        gdf2 (gpd.GeoDataFrame): The second GeoDataFrame.
        overlap_only (bool): Only keep ['Polygon', 'MultiPolygon'] geometries and not ['LineString', 'Point']

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame containing only the areal overlaps,
                           with an 'intersection' column holding the
                           intersection geometry.  Returns an empty
                           GeoDataFrame if no areal overlaps are found.
    """

    if not all(col in gdf1.columns for col in ['geometry', 'cell']):
        raise ValueError("gdf1 must have 'geometry' and 'cell' columns")
    if not all(col in gdf2.columns for col in ['geometry', 'cell']):
        raise ValueError("gdf2 must have 'geometry' and 'cell' columns")


    gdf1['geometry'] = gdf1['geometry'].apply(lambda geom: make_valid(geom))
    gdf2['geometry'] = gdf2['geometry'].apply(lambda geom: make_valid(geom))
        
    # Perform a spatial join to find intersecting features
    df_intersection = gpd.sjoin(gdf1, gdf2, how="inner", predicate="intersects", lsuffix='1', rsuffix='2')

    # Rename cell column in gdf2 to avoid suffix conflicts
    gdf2['cell_2_'] = gdf2['cell']
    df_intersection['cell_2_'] = df_intersection['cell_2']

    # Merge the df_intersection GeoDataFrame with the original gdf2 to include all columns
    df_intersection = pd.merge(df_intersection, gdf2, on=['cell_2_'], suffixes=('_1', '_2'))

    # Calculate the intersection geometry
    df_intersection['intersection'] = df_intersection.apply(lambda row: row['geometry_1'].intersection(row['geometry_2']), axis=1)

    # Filter out non-areal overlaps (LINESTRING, POINT, etc.)
    if overlap_only:
        df_intersection = df_intersection[df_intersection['intersection'].map(lambda g: g.geom_type in ['Polygon', 'MultiPolygon'])]

    # Ensure the result is a GeoDataFrame, even if empty after filtering
    if len(df_intersection) == 0:
        return gpd.GeoDataFrame(geometry=[], crs=gdf1.crs)  # Empty GeoDataFrame with CRS
    else:
        return gpd.GeoDataFrame(df_intersection, geometry='intersection', crs=gdf1.crs) #Ensure that geometry is the intersection column

def plot_geoms(geoms=None,gdf1=None,gdf2=None):
    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_title("Intersections from sjoin")

    # Plot gdf1 geometries (blue)
    if gdf1 is not None:
        for index, row in gdf1.iterrows():
            plot_polygon(ax, row['geometry'], 'blue', alpha=0.1)
    if gdf2 is not None:
        # Plot gdf2 geometries (red)
        for index, row in gdf2.iterrows():
            plot_polygon(ax, row['geometry'], 'red', alpha=0.1)

    # Plot intersections (green)
    if geoms is not None:
        for index, row in geoms.iterrows():
            plot_polygon(ax, row['intersection'], 'green', alpha=.1)

    ax.set_aspect('equal')

    plt.tight_layout()
    plt.show()

# Test data

In [ ]:
data1 = {
    'cell': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'geometry': [
        sg.Polygon([(0, 0), (0, 2), (2, 2), (2, 0)]),  # A
        sg.MultiPolygon([  # B
            ([(3, 3), (3, 5), (5, 5), (5, 3)], []),
            ([(6, 1), (6, 2), (7, 2), (7, 1)], [])
        ]),
        sg.Point(10, 10),  # C - shifted away
        sg.Polygon([(1, 4), (1, 6), (3, 6), (3, 4)]),  # D - shifted
        sg.Polygon([(4, 7), (4, 9), (6, 9), (6, 7)]),  # E - shifted
        sg.Polygon([(7, 10), (7, 12), (9, 12), (9, 10)]), # F - shifted
        sg.Polygon([(0, 7), (0, 9), (2, 9), (2, 7)]),  # G - shifted
        sg.Polygon([(3, 1), (3, 3), (5, 3), (5, 1)]),  # H - shifted
        sg.Polygon([(6, 4), (6, 6), (8, 6), (8, 4)]),  # I
        sg.Polygon([(0, 10), (0, 12), (2, 12), (2, 10)])   # J - shifted *CRITICAL SHIFT*
    ]
}
gdf1 = gpd.GeoDataFrame(data1, crs="EPSG:4326")  # Add a CRS

data2 = {
    'cell': ['X', 'Y', 'Z'],
    'geometry': [
        sg.Polygon([(1, 1), (1, 3), (3, 3), (3, 1)]),
        sg.Polygon([(4, 4), (4, 6), (6, 6), (6, 4)]),
        sg.Polygon([(7, 7), (7, 9), (9, 9), (9, 7)])
    ]
}
gdf2 = gpd.GeoDataFrame(data2, crs="EPSG:4326")  # Add a CRS


df_intersection = intersecting_geometries(gdf1, gdf2)
df_intersection_shapely = intersecting_geometries_shapely(gdf1, gdf2)

plot_geoms(df_intersection,gdf1=gdf1,gdf2=gdf2)
plot_geoms(df_intersection_shapely,gdf1=gdf1,gdf2=gdf2)

# Segmentation data

In [2]:
f1 = '/work/PRTNR/CHUV/DIR/rgottar1/single_cell_all/users/skang/Programming/img_registration/ihc_seg/transformed_coords/10x_5um_lung_0PSV.geojson.gz'
f2 = '/work/PRTNR/CHUV/DIR/rgottar1/spatial/data/xenium_paper/xenium/processed/segmentation/proseg/NSCLC/lung/0PSV/0PSV/raw_results/cell-polygons.geojson.gz'

gdf1 = gpd.read_file("gzip://" +f1)
gdf2 = gpd.read_file("gzip://" +f2)

if 'cell' not in gdf1:
    gdf1['cell']=gdf1['id']
if 'cell' not in gdf2:
    gdf1['cell']=gdf1['id']

gdf2 = gdf2
gdf1 = gdf1

In [15]:
%time df_intersection = intersecting_geometries(gdf1, gdf2)
%time df_intersection_shapely = intersecting_geometries_shapely(gdf1, gdf2)
# plot_geoms(df_intersection,gdf1=gdf1,gdf2=gdf2)
# plot_geoms(df_intersection_shapely,gdf1=gdf1,gdf2=gdf2)

CPU times: user 9.71 s, sys: 131 ms, total: 9.84 s
Wall time: 9.93 s
CPU times: user 11.5 s, sys: 150 ms, total: 11.7 s
Wall time: 11.8 s


In [ ]:
df_overlay = gdf1.overlay(gdf2, how='intersection')
df_sjoin_nearest_gdf1 = gdf1.sjoin_nearest(gdf2,max_distance=None,distance_col='distance')
df_sjoin_nearest_gdf2 = gdf2.sjoin_nearest(gdf1,max_distance=None,distance_col='distance')

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/geopandas/geodataframe.py:2675: UserWarning: `keep_geom_type=True` in overlay resulted in 1 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,id,name,objectType,geometry,cell_left,index_right,cell_right,cell_2_,distance
0,bd5eb9d9-141a-4966-ad32-124e7eb3ffc8,aaaamgje-1,detection,"POLYGON ((11366.56892 5048.19838, 11369.99932 ...",bd5eb9d9-141a-4966-ad32-124e7eb3ffc8,72776,72776,72776,6803.620602
1,a47d99fa-d24a-461a-9f3b-ce40193b2693,aaaapjol-1,detection,"POLYGON ((11312.60477 5044.74405, 11318.60344 ...",a47d99fa-d24a-461a-9f3b-ce40193b2693,72776,72776,72776,6763.911018
2,9ef5b676-c49b-4cea-ad21-a1fb0bac5dbf,aaabfddp-1,detection,"POLYGON ((11324.59919 5040.89587, 11325.88434 ...",9ef5b676-c49b-4cea-ad21-a1fb0bac5dbf,72776,72776,72776,6781.018124
3,5a16325c-c321-43b0-9c7a-8dc66fb7df31,aaabfihi-1,detection,"POLYGON ((11340.01782 5041.33198, 11341.73167 ...",5a16325c-c321-43b0-9c7a-8dc66fb7df31,72776,72776,72776,6776.836274
4,c492d9e3-92cf-4008-9ed2-d482ffa9f393,aaacndjo-1,detection,"POLYGON ((11305.33522 5022.46853, 11310.47795 ...",c492d9e3-92cf-4008-9ed2-d482ffa9f393,72776,72776,72776,6742.439875
...,...,...,...,...,...,...,...,...,...
73401,4397b69f-b852-4f60-a8bb-523ae5f2cea0,oiejdgmd-1,detection,"POLYGON ((6966.89024 6202.32638, 6969.4619 619...",4397b69f-b852-4f60-a8bb-523ae5f2cea0,51253,51253,51253,3231.602827
73402,add07967-54f2-479f-9a8c-9236caab59f8,oiejhaoc-1,detection,"POLYGON ((6943.76607 6195.03316, 6945.90869 61...",add07967-54f2-479f-9a8c-9236caab59f8,51253,51253,51253,3204.232102
73403,6eb6103e-5291-427c-b750-64d05a19a1dd,oiekjomk-1,detection,"POLYGON ((6935.21184 6172.32886, 6936.9252 617...",6eb6103e-5291-427c-b750-64d05a19a1dd,51253,51253,51253,3197.235688
73404,c720e58d-103f-4441-9b62-529d9061556d,oielncbc-1,detection,"POLYGON ((6981.03564 6180.06198, 6982.74948 61...",c720e58d-103f-4441-9b62-529d9061556d,51253,51253,51253,3227.408419
